In [10]:
import pandas as pd
import os
import requests
from tqdm import tqdm

In [11]:
# --- Configuration ---
csv_path = "Stars-800.csv"  # CSV file with 'ra' and 'dec' columns
output_dir = "sdss_800_images"  # Output folder for JPEGs
scale = 0.25  # 64.0 arcsec
width = 256  # image width in pixels
height = 256  # image height in pixels

In [12]:
# --- Create output directory ---
os.makedirs(output_dir, exist_ok=True)

In [13]:
# --- Read coordinates ---
df = pd.read_csv(csv_path)
if 'ra' not in df.columns or 'dec' not in df.columns:
    raise ValueError("CSV must contain 'ra' and 'dec' columns")


In [14]:
# --- Function to download image from SDSS ---
def download_sdss_jpeg(ra, dec, filename):
    url = "https://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/getjpeg"
    params = {
        "ra": ra,
        "dec": dec,
        "scale": scale,
        "width": width,
        "height": height,
        "opt": ""  # empty = no overlays (you can try "G" for grid, etc.)
    }
    response = requests.get(url, params=params, timeout=10)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
    else:
        print(f"Failed to download image for RA={ra}, DEC={dec}, Status={response.status_code}")


In [15]:
# --- Loop through coordinates and download images ---
for i, row in tqdm(df.iterrows(), total=len(df)):
    ra = row['ra']
    dec = row['dec']
    filename = os.path.join(output_dir, f"star_{i}_ra{ra:.4f}_dec{dec:.4f}.jpg")
    download_sdss_jpeg(ra, dec, filename)

print(f"Downloaded {len(df)} JPEG images to '{output_dir}'")

100%|██████████| 800/800 [14:18<00:00,  1.07s/it]

Downloaded 800 JPEG images to 'sdss_800_images'
